**version 5.3**

# Generating Poincaré plots

This example shows how to generate Poincaré plots, a.k.a. puncture plots, with ASCOT5. These plots are generated by tracing markers and recording their position each time a marker crosses a poloidal or toroidal plane. They are mainly used to visualize magnetic field structures and particle resonances. Poincare plots can be made for both field lines and particles.

This tutorial consists of three parts:

1. <a href='#walkthrough'>Quick walkthrough</a>
2. <a href='#generating'>Generating the input</a>
3. <a href='#plotting'>Plotting the output</a>

<a id='walkthrough'></a>


## Quick walkthrough

This is a short guide to show how Poincaré plots are generated most of the time.

First we create a simple test case (skip this part when doing actual studies).

In [ ]:
from a5py import Ascot

a5 = Ascot("ascot.h5", create=True)
a5.data.create_input("options tutorial")
a5.data.create_input("bfield analytical iter circular")
a5.data.create_input("wall rectangular")
a5.data.create_input("plasma flat")
a5.data.create_input("E_TC", exyz=np.array([0,0,0]))
a5.data.create_input("N0_3D")
a5.data.create_input("Boozer")
a5.data.create_input("MHD_STAT")
print("Inputs created")

Next we create markers and options for the Poincaré simulation.
There is nothing extraordinary in these inputs: the markers are just initialized at uniform interals in radius and the options disable all other physics except orbit-following, set proper end conditions, and enable Poincaré data collection.

In [ ]:
a5.input_init(bfield=True)
a5.data.create_input("marker poincare", activate=True)
a5.input_free()
print("Marker input written")

In [ ]:
a5.data.create_input("options poincare", maxrho=True, activate=True)
print("Options written")

Now you can either run the simulation using ``ascot5_main`` or, as we do here, utilize the live simulation support.

In [ ]:
a5.simulation_initinputs()
mrk = a5.data.marker.active.read()
a5.simulation_initmarkers(**mrk)
opt = a5.data.options.active.read()
a5.simulation_initoptions(**opt)
vrun = a5.simulation_run(printsummary=True)
print("Done")

The simulations collecting Poincaré data are usually fast to run since they usually contain approx. 100 markers that are traced for approx. 1000 poloidal/toroidal turns.
However, if the simulation lasts long (several tens of minutes) that might indicate there is something wrong with the simulation data

Now we can plot the results.

In [ ]:
vrun.plotorbit_poincare(1, conlen=False)

Remember to free resources if you ran a live simulation.

In [ ]:
a5.simulation_free()

<a id='generating'></a>


## Generating the input

For a Poincaré simulation it is necessary to have a dedicated marker and options input. The easiest way to generate these is to run the script found at `a5py/preprocessing/poincare.py`. The arguments are printed if no arguments are given when running the script.

**fn**  
HDF5 file with a magnetic field present where Poincaré inputs are written. Initializing Poincaré markers requires magnetic field and the one that is set as active is used.

**ntrace, rhomin, and rhomax**  
ntrace is the total number of markers to be simulated and these are initially distributed radially evenly between the interval \[rhomin, rhomax\]. Note that if the magnetic field is poorly normalized (there is a "hole" in rho data on the axis when plotted in GUI) this initialization might fail.

**tor0, pol0**  
Initial toroidal and poloidal angle \[deg\] for the markers. If negative, markers are randomly distributed. Usual habit is to initialize markers at the outer mid-plane (pol0 = 0) and randomly in toroidal direction (tor0 = -1).
 
**rhoend**  
Maximum rho value at which marker simulation is terminated. Negative value means markers are traced until they hit the wall.

**time**  
Time slice at which marker simulation begins. Only relevant for time-dependent fields and MHD. Note that for field line marker simulations time is not evolved so the Poincaré plot is the snap shot of the field. For particles this is not the case as the field evolves in time.

**cputmax**  
Maximum CPU time limit for simulating a single marker. Usually it is not necessary to set this value. However, field line tracing uses adaptive time step (so does the particle tracing but this can be changed manually to the fixed step scheme) and so the simulation might get "stuck" if the field has poor quality.

**pitch**  
Marker pitch. Relevant for when particles are simulated. For field lines this only controls if markers are traced along (1) or opposite (-1) to the field lines. *Note that you will need to change this parameter for particle simulations as the pitch equal to unity (default value) is not acceptable.*

**tor, pol**  
Toroidal angles \[deg\] for (R,z)-planes and poloidal angles for (rho,phi)-planes where Poincaré data is collected. Negative value means that particular Poincaré data is not collected. You can set up multiple planes by giving several angles as `-pol "10 20"`. Unless you are doing something fancy, the default values are ok.

**mhd**  
Toggle whether to include MHD modes or not. Note that the field quality might be poor near the core and the separatrix if the boozer data is of poor quality. Adjusting cputmax, rhomin, and rhomax parameters might still in this case enable you to generate a Poincaré plot.

**species**  
Setting this parameter turns the initialized markers from field line tracers to actual particles which are simulated with the adaptive guiding center mode (it is possible to do gyro-orbit simulation but this has to be changed afterwards in the options). This parameter sets mass and charge(state) for the marker, e.g. setting `-species electron` causes markers to be initialized as electrons. See `a5py/physlib/species.py` for a list of particle species.

**energy**  
Sets the particle energy \[eV\]. Required when particles are simulated instead of field lines.

To include Poincaré input generation to your own script, make the following import:

In [ ]:
from a5py.preprocessing.poincare import gen_poincareoptions, gen_poincaremarkers

These functions generate the data but does not write it to HDF5 (unless `desc` argument is given).

<a id='plotting'></a>


## Plotting the output

While GUI is the most obvious choice for the plotting, you might want to include the plotting to your scripts e.g. to make a publication quality plot. The Poincaré data is stored in the orbit output after the simulation.